In [ ]:
import os
import json
import re
import PyPDF2


In [ ]:
def load_papers_from_jsonl(file_path):
    papers = []
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                try:
                    paper = json.loads(line.strip())
                    papers.append(paper)
                except json.JSONDecodeError:
                    print(f"Skipping a line due to JSON decoding error.")
    except FileNotFoundError:
        print(f"The file {file_path} was not found.")
    except Exception as e:
        print(f"An unexpected error occurred while reading the file: {e}")

    print(f"Loaded papers from {file_path}.")
    return papers

# Specify the path to your JSONL file
jsonl_file_path = "extracted_dfrws_papers_NEWEST_final.jsonl"

# Load papers
papers = load_papers_from_jsonl(jsonl_file_path)

# Display loaded papers (Optional)
for i, paper in enumerate(papers[:5]):  # Limit display to the first 5 papers for readability
    print(f"Paper {i+1}:")
    print(f"Title: {paper.get('title', 'No title provided')}")
    print("Content:")
    print(paper.get('content', 'No content provided')[:500])  # Print first 500 characters of content
    print("-" * 50)  # Separator


In [ ]:
def generate_title_prompt(paper):
    title = paper['title']
    content = paper['content']
    return f'''
    You are tasked with extracting the full title from the digital forensics paper titled "{title}".

    Guidelines:
    - The title is usually at the top of the first page or in the first section.
    - Extract the title in its entirety.

    Your response must be in the following JSON format:
    {{
        "title": "Title of the paper here"
    }}

    Here is the paper content:
    <Start of Paper Content>
    {content}
    <End of Paper Content>

    Your response: """
    '''


def generate_tools_prompt(paper):
    title = paper['title']
    content = paper['content']
    
    return f'''
    You are tasked with extracting **tools** mentioned in the digital forensics paper titled "{title}".

 Guidelines:
 
     \t1. A **tool** is any named software, framework, system, or purpose-built script that is explicitly used or created for forensic or anti-forensic purposes.
        \t- Only include a tool if it is actually used, created, or extended in the paper. 
        \t- As an AI assistant, you must differentiate between tools that are actually **used**, **created**, or **extended** in the paper versus those that are only **referenced** or **mentioned**. 
        \t- Some papers explicitly state phrases like *“we created a tool”* or *“we used a tool.”* 
        \t- However, if a tool is only mentioned for context in sections such as **Related Work**, **Literature Review**, **Background**, or in paper types like **SLR**, **Survey**, or **SoK**, it should NOT be included in the tools output.
        \t- General-purpose programming languages (e.g., Python, Java, C++), machine learning libraries or algorithms (e.g., Random Forest, SVM, TensorFlow, scikit-learn), and build systems (e.g., Ninja, CMake, Make) are NOT considered forensic or anti-forensic tools. 
        \t- EXCLUDE supporting software. The following are NOT forensic/anti-forensic tools unless the paper explicitly presents a purpose-built forensic plugin/module built on top of them (in which case ONLY the plugin/module is the tool, not the platform):
        \t\t-  • Databases: MySQL (https://www.mysql.com/), PostgreSQL (https://www.postgresql.org/), SQLite (https://www.sqlite.org/), MongoDB (https://www.mongodb.com/)
        \t\t-  • Web servers / app servers: Apache HTTP Server (https://httpd.apache.org/), Nginx (https://nginx.org/), Microsoft IIS, Tomcat (https://tomcat.apache.org/)
        \t\t-  • Operating systems: Ubuntu, Debian, Windows, macOS
        \t\t-  • Build/compilers: Ninja (https://ninja-build.org/), CMake (https://cmake.org/), Make, GCC/Clang
        \t\t-  • Languages & ML libs/algorithms: Python, Java, C/C++, R, Random Forest, SVM/SVC, TensorFlow, scikit-learn, PyTorch
        \t\t- URL/Domain rule (strong): If a tool’s URL resolves to one of the above platform vendor domains (e.g., "https://www.mysql.com/" for MySQL, "https://www.sqlite.org/" for SQlite), classify it as supporting software and EXCLUDE it from the tools output (return no entry). Do NOT relabel it as a tool. 


    \t2. For each tool actually **used**, **created**, or **extended**, provide:
        \t- "tool_name"
        \t- "action": "used" or "created" or "extended"
        \t- "repository_link" (URL or empty string)
        \t- "license": must be one of:
          \t\t- "open-source": source code is publicly available, allowing others to inspect, modify, and extend. Open-source tools are continuously updated and widely reused. They enable creativity and expansion since others can build upon the original code. Users can also often download ready-to-run executables in addition to modifying the source. Examples: Kali Linux, CAINE, Autopsy.
          \t\t- "proprietary": source code is closed and controlled by the originator (e.g., company or vendor). Users may download and run the executable (installer, binary, or licensed version), but cannot view or modify the source code. Proprietary tools are widely used in practice, especially in law enforcement, but cannot be extended by the community. Examples: FTK Forensic Toolkit, FTK Imager, Magnet AXIOM (Magnet Forensics).
          \t\t- "not-mentioned": if the license type is not explicitly stated and no reliable source (e.g., URL, DOI) is available.
          \t- **Note:** The license type is independent of origin. Academic research tools can be released as either open-source (e.g., Bulk Extractor) or proprietary (e.g., closed binaries distributed by an academic team). 
          \t- If no evidence is available in either the paper or its referenced sources, return "not-mentioned".

        \t- "origin": one of:
          \t\t- "academic_research_DFRWS" if the tool was first introduced by this DFRWS paper
          \t\t- "academic_research_external" if the tool was created in other academic venues (conferences, journals, academic projects)
          \t\t- "organization" if the tool was created by companies, vendors, or non-academic organizations
          \t\t- "not-mentioned" if the origin is not explicitly stated.

     \t3. Special case:
        \t- If the paper introduces a plugin, module, extension, or significant modification of an existing tool, mark "action" as "extended" and apply the same origin rules.
        \t- Also list the base tool separately if it was explicitly used.


    \t4. If the paper uses **no tools**, return: null

       {{
         "tools": [{{"tool_name": null, "action": null, "repository_link": null, "repository_type": null, "origin": null}}]
       }}
    \t- Example 1: In the paper “Audit Data Reduction Using Neural Networks and Support Vector Machines” by Srinivas Mukkamala and Andrew Sung (DFRWS 2002), the authors used Neural Networks and Support Vector Machines (SVMs),
    referencing the SVMlight implementation. According to the Guidelines, these are general-purpose machine learning libraries/algorithms and 
    not forensic or anti-forensic tools. Since no forensic tool was actually created or used, the expected output is:
    
    {{
      "tools": null
    }}
    \t- Example 2: In the paper “Language and Gender Author Cohort Analysis of E-mail for Computer Forensics” by Olivier de Vel, Malcolm Corney, Alison Anderson, and George Mohay (DFRWS 2002), 
    the authors use Support Vector Machines **SVMlight** for classification on stylometric/structural features. **SVMlight** is a general-purpose ML library, 
    not a forensic tool, so per the Guidelines the output is:

    {{
      "tools": null
    }}
    
    \t5. If a tool is found but any field (action, repository_link, repository_type, or authorship) is not mentioned, assign "not-mentioned" to that field.

    JSON Format:
    {{
      "tools": [
        {{
          "tool_name": "Volatility",
          "action": "used",
          "repository_link": "https://github.com/volatilityfoundation/volatility",
          "repository_type": "open-source",
          "origin": "organization"
        }},
    
        {{
          "tool_name":"DROP",
          "action": "created"
          "repository_link": "https://github.com/unhcfreg/DROP",
          "repository_type": "open-source",
          "origin": "academic_research_DFRWS"
        }}
      ]
    }}
     \t6. For example in the paper "So fresh, so clean: Cloud forensic analysis of the Amazon iRobot Roomba vacuum", DFRWS authors created
    a tool and made it open-source:
    {{
      "tools": [
        {{
          "tool_name": "PyRoomba",
          "action": "created",
          "repository_link": "https://github.com/BiTLab-BaggiliTruthLab/PyRoomba",
          "repository_type": "open-source",
          "origin": "academic_research_DFRWS"
        }}
    }}

    Here is the paper content:
    <Start of Paper Content>
    {content}
    <End of Paper Content>

    Your response: """
    '''


In [ ]:
import pandas as pd

# Read Excel (.xlsx)
df = pd.read_excel("tools_new_expanded.csv.xlsx")  # or rename file to .xlsx if it really is Excel

# ---------- Normalization helpers ----------
def norm(s):
    if pd.isna(s):
        return None
    return str(s).strip()

def norm_lower(s):
    s = norm(s)
    return s.lower() if s is not None else None

def normalize_license(val):
    v = norm_lower(val)
    if v is None or v in {"", "na", "n/a", "none", "null"}:
        return "not-mentioned"
    # collapse variants like "open-source (gplv3)"
    if v.startswith("open-source"):
        return "open-source"
    if v.startswith("proprietary"):
        return "proprietary"
    if v == "not-mentioned":
        return "not-mentioned"
  

def normalize_action(val):
    v = norm_lower(val)
    if v in {"used", "created", "extended"}:
        return v
    return None  # treat anything else as "no tool entry"

def normalize_origin(val):
    v = norm_lower(val)
    allowed = {
        "academic_research_dfrws",
        "academic_research_external",
        "non-academia",
        "not-mentioned",
    }
    if v in allowed:
        return v
    return "not-mentioned"

# ---------- Create normalized columns ----------
df["action_norm"]  = df.get("Action", pd.Series([None]*len(df))).apply(normalize_action)
df["origin_norm"]  = df.get("Origin", pd.Series(["not-mentioned"]*len(df))).apply(normalize_origin)
df["license_norm"] = df.get("License", pd.Series(["not-mentioned"]*len(df))).apply(normalize_license)

# ---------- Helper to pretty print a value_counts dict ----------
def print_counts(title, series):
    print(title)
    vc = series.value_counts()
    for k, v in vc.items():
        print(f"  {k}: {int(v)}")
    if vc.empty:
        print("  (none)")
    print()

# ---------- Global metrics ----------
total_rows = len(df)
null_count = int(df["action_norm"].isna().sum())
used_count     = int((df["action_norm"] == "used").sum())
created_count  = int((df["action_norm"] == "created").sum())
extended_count = int((df["action_norm"] == "extended").sum())

print(f"Total rows (tool entries): {total_rows}")
print(f"Null tool entries (no valid action): {null_count}\n")
print(f"Used tools: {used_count}")
print(f"Created tools: {created_count}")
print(f"Extended tools: {extended_count}\n")

# ---------- Overall distributions ----------
print_counts("Overall license counts (normalized):", df["license_norm"])
print_counts("Overall origin counts (normalized):", df["origin_norm"])

# ---------- Created breakdown ----------
df_created = df[df["action_norm"] == "created"]
print_counts("Created tools by origin:", df_created["origin_norm"])
print_counts("Created tools by license:", df_created["license_norm"])

# ---------- DFRWS-created licenses ----------
df_dfrws_created = df_created[df_created["origin_norm"] == "academic_research_dfrws"]
print_counts("Licenses of DFRWS-created tools (normalized):", df_dfrws_created["license_norm"])

# ---------- USED breakdown (new) ----------
df_used = df[df["action_norm"] == "used"]
print_counts("Used tools by origin:", df_used["origin_norm"])
print_counts("Used tools by license:", df_used["license_norm"])

# ---------- EXTENDED breakdown (optional, symmetry) ----------
df_extended = df[df["action_norm"] == "extended"]
print_counts("Extended tools by origin:", df_extended["origin_norm"])
print_counts("Extended tools by license:", df_extended["license_norm"])


In [ ]:
# -------------------------- Imports & Config --------------------------
import sys, subprocess
from pathlib import Path
import pandas as pd

INPUT_PATH = Path("tools_new_expanded.csv.xlsx")  # your file path
ENCODING = "latin-1"
SHEET_NAME = 0
SAVE_OUTPUTS = False
AUTO_INSTALL_OPENPYXL = True  # set False if you don't want auto-install

# -------------------------- Helpers --------------------------
def ensure_openpyxl():
    try:
        import openpyxl  # noqa: F401
        return True
    except ImportError:
        if not AUTO_INSTALL_OPENPYXL:
            return False
        try:
            print("openpyxl not found. Installing...", flush=True)
            subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl"])
            import openpyxl  # noqa: F401
            print("openpyxl installed.\n", flush=True)
            return True
        except Exception as e:
            print(f"Failed to install openpyxl: {e}", flush=True)
            return False

def load_table(p: Path) -> pd.DataFrame:
    """
    Try CSV first. If it looks like Excel or CSV parse fails,
    try Excel (using openpyxl). Reads everything as string.
    """
    # Try CSV
    try:
        return pd.read_csv(p, encoding=ENCODING, dtype=str)
    except Exception as csv_err:
        # Fall back to Excel if CSV failed
        ok = ensure_openpyxl()
        if not ok:
            raise ImportError(
                "Missing 'openpyxl' needed to read Excel. "
                "Install it via: pip install openpyxl"
            ) from csv_err
        return pd.read_excel(p, sheet_name=SHEET_NAME, dtype=str)

def norm(s):
    if pd.isna(s):
        return None
    return str(s).strip()

def norm_lower(s):
    s = norm(s)
    return s.lower() if s is not None else None

def normalize_license(val):
    v = norm_lower(val)
    if v is None or v in {"", "na", "n/a", "none", "null"}:
        return "not-specified"
    if v.startswith("open-source"):
        return "open-source"
    if v.startswith("proprietary"):
        return "proprietary"
    if v == "not-specified":
        return "not-specified"
    if any(x in v for x in ["gpl", "mit", "apache", "bsd", "lgpl", "agpl"]):
        return "open-source"
    return "not-specified"

def normalize_action(val):
    v = norm_lower(val)
    return v if v in {"used", "created", "extended"} else None

def normalize_origin(val):
    v = norm_lower(val)
    allowed = {
        "academic_research_dfrws",
        "academic_research_external",
        "non-academia",
        "not-specified",
    }
    return v if v in allowed else "not-specified"

def map_case_insensitive(df: pd.DataFrame):
    seen = {}
    for c in df.columns:
        cl = c.strip().lower()
        if cl not in seen:
            seen[cl] = c
    return seen

def get_col(df: pd.DataFrame, colmap: dict, name: str):
    return colmap.get(name.strip().lower())

def print_table(title, table):
    print(f"\n{title}")
    print(table.to_string())

def print_counts(title, series: pd.Series):
    print(f"\n{title}")
    counts = series.value_counts(dropna=False)
    for k, v in counts.items():
        label = "NaN" if pd.isna(k) else str(k)
        print(f"  {label}: {int(v)}")

# -------------------------- Main --------------------------
def main():
    df = load_table(INPUT_PATH)

    # Clean header names and map case-insensitively
    df.columns = [c.strip() for c in df.columns]
    colmap = map_case_insensitive(df)

    action_col  = get_col(df, colmap, "Action")
    origin_col  = get_col(df, colmap, "Origin")
    license_col = get_col(df, colmap, "License")

    # Fallback columns if missing
    if action_col is None:
        df["__Action__"] = None
        action_col = "__Action__"
    if origin_col is None:
        df["__Origin__"] = "not-specified"
        origin_col = "__Origin__"
    if license_col is None:
        df["__License__"] = "not-specified"
        license_col = "__License__"

    # Normalized columns
    df["action_norm"]  = df[action_col].apply(normalize_action)
    df["origin_norm"]  = df[origin_col].apply(normalize_origin)
    df["license_norm"] = df[license_col].apply(normalize_license)

    # Global stats
    total_rows = len(df)
    null_count = int(df["action_norm"].isna().sum())
    df_act = df[df["action_norm"].notna()].copy()
    total_action_instances = len(df_act)

    print(f"Total tool entries (paper–tool instances): {total_rows}")
    print(f"Null tool entries (no valid action): {null_count}\n")

    # Overall action breakdown
    print_counts("Overall license counts (normalized):", df["license_norm"])
    print_counts("Overall origin counts (normalized):",  df["origin_norm"])

    # Action-specific subsets
    df_used    = df[df["action_norm"] == "used"]
    df_created = df[df["action_norm"] == "created"]
    df_extended= df[df["action_norm"] == "extended"]

    print(f"\nUsed tools: {len(df_used)}")
    print(f"Created tools: {len(df_created)}")
    print(f"Extended tools: {len(df_extended)}")

    # Created breakdowns
    print_counts("Created tools by origin:",  df_created["origin_norm"])
    print_counts("Created tools by license:", df_created["license_norm"])

    df_dfrws_created = df_created[df_created["origin_norm"] == "academic_research_dfrws"]
    print_counts("Licenses of DFRWS-created tools (normalized):", df_dfrws_created["license_norm"])

    # Used breakdowns
    print_counts("Used tools by origin:",  df_used["origin_norm"])
    print_counts("Used tools by license:", df_used["license_norm"])

    # Extended breakdowns
    print_counts("Extended tools by origin:",  df_extended["origin_norm"])
    print_counts("Extended tools by license:", df_extended["license_norm"])

    # Pivot tables
    action_cols = ["used", "created", "extended"]
    if total_action_instances == 0:
        print("\nNo valid action rows found. Check the 'Action' column values.")
        return

    license_action = pd.crosstab(
        df_act["license_norm"], df_act["action_norm"], dropna=False
    ).reindex(columns=action_cols, fill_value=0)
    license_action.loc["TOTAL"] = license_action.sum()
    license_action["TOTAL"] = license_action.sum(axis=1)

    origin_action = pd.crosstab(
        df_act["origin_norm"], df_act["action_norm"], dropna=False
    ).reindex(columns=action_cols, fill_value=0)
    origin_action.loc["TOTAL"] = origin_action.sum()
    origin_action["TOTAL"] = origin_action.sum(axis=1)

    print_table("License × Action counts", license_action)
    print_table("Origin × Action counts", origin_action)

    if SAVE_OUTPUTS:
        license_action.to_csv("license_x_action_counts.csv", index=True)
        origin_action.to_csv("origin_x_action_counts.csv", index=True)
        print("\nSaved: license_x_action_counts.csv, origin_x_action_counts.csv")

if __name__ == "__main__":
    main()


In [ ]:
"""
Frequency of widely used tools from tools_new_expanded.csv.xlsx

What it does:
- Reads the Excel file (expects columns: Title, Tool Name, Action, Repository Link, License, Origin)
- Normalizes tool names (trim whitespace; optional lowercasing)
- Computes:
  1) Overall tool frequency (all actions)
  2) "Widely used" tool frequency (Action == "used")
  3) (Optional) Frequency by action type
- Saves results to CSVs and prints top N to console
"""

from pathlib import Path
import pandas as pd

# ----------------------------
# Config
# ----------------------------
INPUT_PATH = Path("tools_new_expanded.csv.xlsx")  # adjust path if needed
SHEET_NAME = 0  # or set to a sheet name string
TOP_N = 30

# If you want case-insensitive merging of tool names, set True
LOWERCASE_TOOLNAMES = False

# ----------------------------
# Load
# ----------------------------
df = pd.read_excel(INPUT_PATH, sheet_name=SHEET_NAME)

# Ensure expected columns exist (rename here if your headers differ slightly)
expected = ["Title", "Tool Name", "Action", "Repository Link", "License", "Origin"]
missing = [c for c in expected if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}\nFound: {list(df.columns)}")

# ----------------------------
# Clean / normalize
# ----------------------------
df["Tool Name"] = df["Tool Name"].astype("string").str.strip()
df["Action"] = df["Action"].astype("string").str.strip().str.lower()

# Drop null/empty tool names
df = df[df["Tool Name"].notna() & (df["Tool Name"] != "")].copy()

# Optional: normalize tool names to lower-case for grouping
if LOWERCASE_TOOLNAMES:
    df["Tool Name Norm"] = df["Tool Name"].str.lower()
else:
    df["Tool Name Norm"] = df["Tool Name"]

# ----------------------------
# 1) Overall frequency (all actions)
# ----------------------------
overall = (
    df.groupby("Tool Name Norm", dropna=False)
      .size()
      .reset_index(name="count")
      .sort_values(["count", "Tool Name Norm"], ascending=[False, True])
)

# ----------------------------
# 2) Widely used = Action == "used"
# ----------------------------
used_df = df[df["Action"] == "used"].copy()

used_freq = (
    used_df.groupby("Tool Name Norm", dropna=False)
          .size()
          .reset_index(name="count_used")
          .sort_values(["count_used", "Tool Name Norm"], ascending=[False, True])
)

# ----------------------------
# 3) Frequency by action (used/created/extended) per tool
# ----------------------------
by_action = (
    df.pivot_table(
        index="Tool Name Norm",
        columns="Action",
        values="Title",
        aggfunc="size",
        fill_value=0,
        dropna=False,
    )
    .reset_index()
)

# Add totals and sort by "used" first if present
if "used" in by_action.columns:
    by_action["TOTAL"] = by_action.drop(columns=["Tool Name Norm"]).sum(axis=1)
    by_action = by_action.sort_values(["used", "TOTAL", "Tool Name Norm"], ascending=[False, False, True])
else:
    by_action["TOTAL"] = by_action.drop(columns=["Tool Name Norm"]).sum(axis=1)
    by_action = by_action.sort_values(["TOTAL", "Tool Name Norm"], ascending=[False, True])

# ----------------------------
# Output
# ----------------------------
out_dir = Path("tool_frequency_outputs")
out_dir.mkdir(exist_ok=True)

overall_path = out_dir / "tool_frequency_overall.csv"
used_path = out_dir / "tool_frequency_used_only.csv"
by_action_path = out_dir / "tool_frequency_by_action.csv"

overall.to_csv(overall_path, index=False)
used_freq.to_csv(used_path, index=False)
by_action.to_csv(by_action_path, index=False)

print("\nTop tools (overall):")
print(overall.head(TOP_N).to_string(index=False))

print("\nTop tools (Action == 'used'):")
print(used_freq.head(TOP_N).to_string(index=False))

print(f"\nSaved:\n- {overall_path}\n- {used_path}\n- {by_action_path}")

# ----------------------------
# Optional: quick bar chart for top used tools
# Uncomment if you want a plot saved as PNG
# ----------------------------
# import matplotlib.pyplot as plt
# top_used = used_freq.head(TOP_N)
# plt.figure()
# plt.bar(top_used["Tool Name Norm"], top_used["count_used"])
# plt.xticks(rotation=75, ha="right")
# plt.ylabel("Count (used)")
# plt.title(f"Top {TOP_N} Widely Used Tools (Action='used')")
# plt.tight_layout()
# plot_path = out_dir / f"top_{TOP_N}_used_tools.png"
# plt.savefig(plot_path, dpi=200)
# print(f"Saved plot: {plot_path}")


In [ ]:
"""
Complete script: Overall tool frequency (ALL actions) + bar chart with license legend
Fixes the "nan tool" issue by removing missing/blank tool names BEFORE string conversion.

Input:  tools_new_expanded.csv.xlsx
Cols:   Title, Tool Name, Action, Repository Link, License, Origin
Output:
  - Prints top N tools (overall frequency)
  - Displays bar chart (colored by license)
  - Saves cleaned frequency table + chart to disk
"""

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# ----------------------------
# Config
# ----------------------------
INPUT_FILE = Path("tools_new_expanded.csv.xlsx")
TOP_N = 15

OUT_DIR = Path("tool_frequency_outputs")
OUT_DIR.mkdir(exist_ok=True)

FREQ_CSV = OUT_DIR / "tool_frequency_overall.csv"
PLOT_PNG = OUT_DIR / f"top_{TOP_N}_tools_overall_by_license.png"

LICENSE_COLORS = {
    "open-source": "#4daf4a",     # green
    "proprietary": "#e41a1c",     # red
    "not-specified": "#999999",   # gray
}

# Canonical tool-name normalization rules
# (Add more ecosystems here as needed)
TOOL_NORMALIZATION = [
    # Normalize all Cellebrite variants to an ecosystem label
    (r"\bcellebrite\b", "Cellebrite UFED (ecosystem)"),
]

# ----------------------------
# Load
# ----------------------------
df = pd.read_excel(INPUT_FILE)

# ----------------------------
# Clean (IMPORTANT: drop NaN tool names before converting to string)
# ----------------------------
df = df[df["Tool Name"].notna()].copy()

df["Tool Name"] = df["Tool Name"].astype("string").str.strip()
df["Action"] = df["Action"].astype("string").str.lower().str.strip()
df["License"] = df["License"].astype("string").str.lower().str.strip()

# Drop empty tool names and any accidental "nan" strings
df = df[df["Tool Name"].notna() & (df["Tool Name"] != "")]
df = df[df["Tool Name"].str.lower() != "nan"]

# Exclude DB Browser for SQLite
df = df[df["Tool Name"].str.lower() != "db browser for sqlite"]

# ----------------------------
# Normalize tool names (ecosystem-level)
# ----------------------------
for pattern, canonical in TOOL_NORMALIZATION:
    df.loc[df["Tool Name"].str.contains(pattern, case=False, na=False, regex=True), "Tool Name"] = canonical

# ----------------------------
# Normalize License values (optional safety)
# ----------------------------
df["License"] = df["License"].fillna("not-specified")
df.loc[~df["License"].isin(LICENSE_COLORS.keys()), "License"] = "not-specified"

# ----------------------------
# Overall frequency (all actions)
# ----------------------------
freq = (
    df.groupby(["Tool Name", "License"], dropna=False)
      .size()
      .reset_index(name="count")
      .sort_values(["count", "Tool Name"], ascending=[False, True])
)

# Save frequency table
freq.to_csv(FREQ_CSV, index=False)

# ----------------------------
# Display table (top N)
# ----------------------------
print(f"\nTop {TOP_N} tools by overall frequency (all actions):\n")
print(freq.head(TOP_N).to_string(index=False))

# ----------------------------
# Plot (top N)
# ----------------------------
top = freq.head(TOP_N).copy()
colors = top["License"].map(lambda x: LICENSE_COLORS.get(x, LICENSE_COLORS["not-specified"]))

plt.figure(figsize=(11, 6))
plt.bar(top["Tool Name"], top["count"], color=colors)
plt.xticks(rotation=70, ha="right")
plt.ylabel("Frequency (all actions)")
plt.title(f"Top {TOP_N} Tools by Overall Frequency (colored by license)")

# Legend
legend_handles = [
    plt.Line2D([0], [0], color=c, lw=6, label=l.replace("-", " ").title())
    for l, c in LICENSE_COLORS.items()
]
plt.legend(handles=legend_handles, title="License", frameon=False)

plt.tight_layout()
plt.savefig(PLOT_PNG, dpi=300)
plt.show()

print(f"\nSaved:\n- Frequency table: {FREQ_CSV}\n- Plot: {PLOT_PNG}")
